# ENSO

This notebook provides an overview of running the [ENSO Metrics package](https://github.com/CLIVAR-PRP/ENSO_metrics) through the PMP. More information can be found in the [README]( ). Example parameter files are located in the [PMP sample setups]( ). 

**Reference**: Planton, Y., E. Guilyardi, A. T. Wittenberg, J. Lee, P. J. Gleckler, T. Bayr, S. McGregor, M. J. McPhaden, S. Power, R. Roehrig, A. Voldoire, 2020: Evaluating El Niño in climate models with the CLIVAR 2020 ENSO metrics package. Bulletin of the American Meteorological Society. [doi: 10.1175/BAMS-D-19-0337.1](https://doi.org/10.1175/BAMS-D-19-0337.1)

Description for individual metrics can be found at https://github.com/CLIVAR-PRP/ENSO_metrics/wiki.

## Download demo data

The ENSO metrics demo requires downloading a large sample data set (size 10.8 GB). The ENSO metric requires a different set of sample data than the rest of the PMP metrics. This section of the notebook will download that data to your chosen location and generate a basic parameter file.

In [1]:
# Lets get the file containing the data needed for this demo
import requests
#r = requests.get("https://pcmdiweb.llnl.gov/pss/pmpdata/pmp_enso_tutorial_files.txt")
r = requests.get("https://pcmdiweb.llnl.gov/pss/pmpdata/pmp_enso_tutorial_files.v20210823.txt")
with open("enso_data_files.txt","wb") as f:
    f.write(r.content)

If you want to change the location where the demo data and output are stored, you can do so here:

In [2]:
# This is where you will be downloading the sample_data
demo_data_directory = "demo_data_enso_new_obs"
# this line is where your output will be stored
demo_output_directory = "demo_output_enso_new_obs"

Then download the data. The total sample data size is 10.8 GB. This will take several minutes.

In [3]:
# Let's download the files
import cdat_info
try:
    cdat_info.download_sample_data_files("enso_data_files.txt", demo_data_directory)
    print("All files downloaded")
except:
    print("Download failed")

Downloading: 'obs4MIPs_PCMDI_monthly/ECMWF/ERA-20C/mon/psl/gn/v20210727/psl_mon_ERA-20C_PCMDI_gn_190001-201012.nc' from 'https://pcmdiweb.llnl.gov/pss/pmpdata/' in: demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly/ECMWF/ERA-20C/mon/psl/gn/v20210727/psl_mon_ERA-20C_PCMDI_gn_190001-201012.nc
Downloading: 'obs4MIPs_PCMDI_monthly/ECMWF/ERA-20C/mon/ts/gn/v20210727/ts_mon_ERA-20C_PCMDI_gn_190001-201012.nc' from 'https://pcmdiweb.llnl.gov/pss/pmpdata/' in: demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly/ECMWF/ERA-20C/mon/ts/gn/v20210727/ts_mon_ERA-20C_PCMDI_gn_190001-201012.nc
Downloading: 'obs4MIPs_PCMDI_monthly/NOAA-ESRL-PSD/20CR/mon/psl/gn/v20210727/psl_mon_20CR_PCMDI_gn_187101-201212.nc' from 'https://pcmdiweb.llnl.gov/pss/pmpdata/' in: demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly/NOAA-ESRL-PSD/20CR/mon/psl/gn/v20210727/psl_mon_20CR_PCMDI_gn_187101-201212.nc
Downloading: 'obs4MIPs_PCMDI_monthly/NOAA-ESRL-PSD/20CR/mon/ts/gn/v20210727/ts_mon_20CR_PCMDI_gn_187101-201212.nc' from 'https://pcmdiwe

Downloading: 'obs4MIPs_PCMDI_monthly/ECMWF/ERA-INT/mon/ua/gn/v20210727/ua_mon_ERA-INT_PCMDI_gn_198901-201001.nc' from 'https://pcmdiweb.llnl.gov/pss/pmpdata/' in: demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly/ECMWF/ERA-INT/mon/ua/gn/v20210727/ua_mon_ERA-INT_PCMDI_gn_198901-201001.nc
Downloading: 'obs4MIPs_PCMDI_monthly/ECMWF/ERA-INT/mon/uas/gn/v20210727/uas_mon_ERA-INT_PCMDI_gn_197901-201903.nc' from 'https://pcmdiweb.llnl.gov/pss/pmpdata/' in: demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly/ECMWF/ERA-INT/mon/uas/gn/v20210727/uas_mon_ERA-INT_PCMDI_gn_197901-201903.nc
Downloading: 'obs4MIPs_PCMDI_monthly/ECMWF/ERA-INT/mon/va/gn/v20210727/va_mon_ERA-INT_PCMDI_gn_198901-201001.nc' from 'https://pcmdiweb.llnl.gov/pss/pmpdata/' in: demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly/ECMWF/ERA-INT/mon/va/gn/v20210727/va_mon_ERA-INT_PCMDI_gn_198901-201001.nc
Downloading: 'obs4MIPs_PCMDI_monthly/ECMWF/ERA-INT/mon/vas/gn/v20210727/vas_mon_ERA-INT_PCMDI_gn_197901-201903.nc' from 'https://pcmdiweb.llnl.g

After downloading the data, we generate the parameter file for this demo.

In [4]:
from download_sample_data import generate_parameter_files
filenames=["basic_enso_param.py.in"]
generate_parameter_files(demo_data_directory, demo_output_directory, filenames=filenames)

Preparing parameter file: basic_enso_param.py
Saving User Choices


## Usage

The ENSO driver can be run from the command line as `enso_driver.py`. In this notebook, we will use bash cell magic (cells beginning with `%%bash`) to run the ENSO driver as a subprocess.

For help, type:  
```
enso_driver.py --help
```

In [5]:
%%bash
enso_driver.py --help

usage: enso_driver.py [-h] [--parameters PARAMETERS]
                      [--diags OTHER_PARAMETERS [OTHER_PARAMETERS ...]]
                      [--mip MIP] [--exp EXP] [--modpath MODPATH]
                      [--modpath_lf MODPATH_LF]
                      [--modnames MODNAMES [MODNAMES ...]] [-r REALIZATION]
                      [--reference_data_path REFERENCE_DATA_PATH]
                      [--reference_data_lf_path REFERENCE_DATA_LF_PATH]
                      [--metricsCollection METRICSCOLLECTION]
                      [--json_name JSON_NAME] [--netcdf_name NETCDF_NAME]
                      [--results_dir RESULTS_DIR] [--case_id CASE_ID]
                      [--obs_catalogue OBS_CATALOGUE]
                      [--obs_cmor_path OBS_CMOR_PATH] [-d [DEBUG]]
                      [--obs_cmor [OBS_CMOR]] [--nc_out [NC_OUT]]

optional arguments:
  -h, --help            show this help message and exit
  --parameters PARAMETERS, -p PARAMETERS
  --diags OTHER_PARAMETERS [OTHER_PA

### Basic example

Parameters for the ENSO Metrics can be set on the command line or using a parameter file. This first example will use a parameter file, which is shown below.

In [9]:
with open("basic_enso_param.py") as f:
    print(f.read())

import os

#
#  OPTIONS ARE SET BY USER IN THIS FILE AS INDICATED BELOW BY:
#
#

# MODELS
modnames = ['ACCESS1-0']
mip = 'cmip5'  # cmip5, cmip6
exp = 'historical'  # historical, piControl
realization = 'r1i1p1'
modpath = 'demo_data_enso_new_obs/CMIP5_demo_data/%(variable)_Amon_%(model)_historical_%(realization)_185001-200512.nc'
modpath_lf = 'demo_data_enso_new_obs/CMIP5_demo_data/sftlf_fx_%(model)_amip_r0i0p0.nc'

# OBSERVATIONS
obs_cmor = True
obs_cmor_path = "demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly"
obs_catalogue = "demo_data/obs4MIPs_PCMDI-CEM2021_monthly_bySource_catalogue_v20210805_demo.json"

# METRICS COLLECTION
metricsCollection = 'ENSO_perf'  # ENSO_perf, ENSO_tel, ENSO_proc

# OUTPUT
case_id = 'basicTestEnso'
results_dir = os.path.join('demo_output_enso_new_obs',case_id, metricsCollection)

json_name = '%(mip)_%(exp)_%(metricsCollection)_%(case_id)_%(model)_%(realization)'
netcdf_name = json_name
nc_out = False



The next cell runs the ENSO driver using the basic parameter file. This may take several minutes.

In [7]:
%%bash
enso_driver.py -p basic_enso_param.py

/opt/anaconda3/envs/pmp_20220321/lib/python3.9/site-packages/cdms2/MV2.py:318: Warning: arguments order for compress function has changed
it is now: MV2.copmress(array,condition), if your code seems to not react or act wrong to a call to compress, please check this
  warnings.warn(
INFO::2022-03-30 10:20::pcmdi_metrics:: Results saved to a json file: /Users/lee1043/Documents/Research/git/pcmdi_metrics_20220106_2/pcmdi_metrics/doc/jupyter/Demo/demo_output_enso_new_obs/basicTestEnso/ENSO_perf/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json
INFO::2022-03-30 10:21::pcmdi_metrics:: Results saved to a json file: /Users/lee1043/Documents/Research/git/pcmdi_metrics_20220106_2/pcmdi_metrics/doc/jupyter/Demo/demo_output_enso_new_obs/basicTestEnso/ENSO_perf/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json


mip: cmip5
exp: historical
models: ['ACCESS1-0']
realization:  r1i1p1
mc_name: ENSO_perf
outdir: demo_output_enso_new_obs/basicTestEnso/ENSO_perf
netcdf_path: demo_output_enso_new_obs/basicTestEnso/ENSO_perf
debug: False
obs_cmor: True
obs_cmor_path: demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly
egg_pth: /opt/anaconda3/envs/pmp_20220321/share/pmp
output directory for graphics:demo_output_enso_new_obs/basicTestEnso/ENSO_perf
output directory for diagnostic_results:demo_output_enso_new_obs/basicTestEnso/ENSO_perf
output directory for metrics_results:demo_output_enso_new_obs/basicTestEnso/ENSO_perf
list_variables: ['pr', 'sst', 'taux']
list_obs: ['AVISO-1-0', 'ERA-INT', 'GPCP-2-3', 'HadISST-1-1']
PMPdriver: dict_obs readin end
Process start: Wed Mar 30 10:14:06 2022
models: ['ACCESS1-0']
 ----- model:  ACCESS1-0  ---------------------
PMPdriver: var loop start for model  ACCESS1-0
realization: r1i1p1
 --- run:  r1i1p1  ---
 --- var:  pr  ---
var_in_file: pr
var, areacell_in_file, realm: p

               EnsoUvcdatToolsLib AverageMeridional
     ComputeMetric: oneVarRMSmetric, BiasSstLonRmse = ACCESS1-0_r1i1p1 and HadISST
                         NOTE: Estimated landmask applied
               EnsoUvcdatToolsLib AverageMeridional
               EnsoUvcdatToolsLib AverageMeridional
     ComputeCollection: metric = BiasTauxLonRmse
     ComputeMetric: oneVarRMSmetric, BiasTauxLonRmse = ACCESS1-0_r1i1p1 and ERA-Interim
               EnsoUvcdatToolsLib AverageHorizontal
                         NOTE: Estimated landmask applied
               EnsoUvcdatToolsLib AverageHorizontal
                         NOTE: Estimated landmask applied
               EnsoUvcdatToolsLib AverageMeridional
               EnsoUvcdatToolsLib AverageMeridional
     ComputeCollection: metric = EnsoAmpl
     ComputeMetric: oneVarmetric = ACCESS1-0_r1i1p1
     ComputeMetric: oneVarmetric = ERA-Interim
                         NOTE: Estimated landmask applied
     ComputeMetric: oneVarmetric = HadISST


This run saved metrics to two files:  
basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json   basicTestEnso/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json

diveDown metrics are not available in all cases. 

The results section of cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json is shown below.

In [8]:
import json
metrics_file=demo_output_directory+"/basicTestEnso/ENSO_perf/cmip5_historical_ENSO_perf_basicTestEnso_ACCESS1-0_r1i1p1.json"
with open(metrics_file) as f:
    results = json.load(f)["RESULTS"]["model"]["ACCESS1-0"]["r1i1p1"]["value"]
print(json.dumps(results, indent = 2))

{
  "BiasPrLatRmse": {
    "diagnostic": {
      "ACCESS1-0_r1i1p1": {
        "value": null,
        "value_error": null
      },
      "ERA-Interim": {
        "value": null,
        "value_error": null
      },
      "GPCPv2.3": {
        "value": null,
        "value_error": null
      }
    },
    "metric": {
      "ERA-Interim": {
        "value": 1.107530462384606,
        "value_error": null
      },
      "GPCPv2.3": {
        "value": 1.9301242972172135,
        "value_error": null
      }
    }
  },
  "BiasPrLonRmse": {
    "diagnostic": {
      "ACCESS1-0_r1i1p1": {
        "value": null,
        "value_error": null
      },
      "ERA-Interim": {
        "value": null,
        "value_error": null
      },
      "GPCPv2.3": {
        "value": null,
        "value_error": null
      }
    },
    "metric": {
      "ERA-Interim": {
        "value": 0.6464917771721341,
        "value_error": null
      },
      "GPCPv2.3": {
        "value": 1.4165839641155153,
        "value_e

### ENSO Metrics Collections

There are 3 metrics collections available:  
ENSO_perf  
ENSO_tel  
ENSO_proc  

They can be selected using the `--metricsCollection` flag. The first example used the "ENSO_perf" collection.

The next example runs the teleconnection collection. To save individual metrics in netCDF format, it uses the `--nc_out` flag.

In [9]:
%%bash -s "$demo_output_directory"
enso_driver.py -p basic_enso_param.py \
--metricsCollection ENSO_tel \
--results_dir $1/basicTestEnso/ENSO_tel \
--nc_out True

/opt/anaconda3/envs/pmp_20220321/lib/python3.9/site-packages/cdms2/MV2.py:318: Warning: arguments order for compress function has changed
it is now: MV2.copmress(array,condition), if your code seems to not react or act wrong to a call to compress, please check this
  warnings.warn(
INFO::2022-03-30 10:36::pcmdi_metrics:: Results saved to a json file: /Users/lee1043/Documents/Research/git/pcmdi_metrics_20220106_2/pcmdi_metrics/doc/jupyter/Demo/demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1.json
INFO::2022-03-30 10:36::pcmdi_metrics:: Results saved to a json file: /Users/lee1043/Documents/Research/git/pcmdi_metrics_20220106_2/pcmdi_metrics/doc/jupyter/Demo/demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json


mip: cmip5
exp: historical
models: ['ACCESS1-0']
realization:  r1i1p1
mc_name: ENSO_tel
outdir: demo_output_enso_new_obs/basicTestEnso/ENSO_tel
netcdf_path: demo_output_enso_new_obs/basicTestEnso/ENSO_tel
debug: False
obs_cmor: True
obs_cmor_path: demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly
egg_pth: /opt/anaconda3/envs/pmp_20220321/share/pmp
output directory for graphics:demo_output_enso_new_obs/basicTestEnso/ENSO_tel
output directory for diagnostic_results:demo_output_enso_new_obs/basicTestEnso/ENSO_tel
output directory for metrics_results:demo_output_enso_new_obs/basicTestEnso/ENSO_tel
list_variables: ['pr', 'sst']
list_obs: ['AVISO-1-0', 'ERA-INT', 'GPCP-2-3', 'HadISST-1-1']
PMPdriver: dict_obs readin end
Process start: Wed Mar 30 10:22:27 2022
models: ['ACCESS1-0']
 ----- model:  ACCESS1-0  ---------------------
PMPdriver: var loop start for model  ACCESS1-0
realization: r1i1p1
 --- run:  r1i1p1  ---
 --- var:  pr  ---
var_in_file: pr
var, areacell_in_file, realm: pr areacella at

                         NOTE: Estimated landmask applied
                         NOTE: Estimated landmask applied
                         NOTE: Estimated landmask applied
     ComputeMetric: twoVarRMSmetric, EnsoPrMapJja = ACCESS1-0_r1i1p1 and HadISST_GPCPv2.3
                         NOTE: Estimated landmask applied
                         NOTE: Estimated landmask applied
                         NOTE: Estimated landmask applied
     ComputeCollection: metric = EnsoSeasonality
     ComputeMetric: oneVarmetric = ACCESS1-0_r1i1p1
               EnsoUvcdatToolsLib AverageMeridional
               EnsoUvcdatToolsLib AverageMeridional
               EnsoUvcdatToolsLib AverageMeridional
     ComputeMetric: oneVarmetric = ERA-Interim
                         NOTE: Estimated landmask applied
                         NOTE: Estimated landmask applied
                         NOTE: Estimated landmask applied
               EnsoUvcdatToolsLib AverageMeridional
               EnsoUvcdatToolsLi

All of the results (netCDF and JSON) are located in the output directory, which uses the metrics collection name.

In [10]:
!ls {demo_output_directory + "/basicTestEnso/ENSO_tel/*.nc"}

demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoAmpl.nc
demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoPrMapDjf.nc
demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoPrMapJja.nc
demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoSeasonality.nc
demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoSstLonRmse.nc
demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoSstMapDjf.nc
demo_output_enso_new_obs/basicTestEnso/ENSO_tel/cmip5_historical_ENSO_tel_basicTestEnso_ACCESS1-0_r1i1p1_EnsoSstMapJja.nc


Finally, this example runs the remaining metrics collection ENSO_proc:

In [11]:
%%bash -s "$demo_output_directory"
enso_driver.py -p basic_enso_param.py \
--metricsCollection ENSO_proc \
--results_dir $1/basicTestEnso/ENSO_proc

/opt/anaconda3/envs/pmp_20220321/lib/python3.9/site-packages/cdms2/MV2.py:318: Warning: arguments order for compress function has changed
it is now: MV2.copmress(array,condition), if your code seems to not react or act wrong to a call to compress, please check this
  warnings.warn(


mip: cmip5
exp: historical
models: ['ACCESS1-0']
realization:  r1i1p1
mc_name: ENSO_proc
outdir: demo_output_enso_new_obs/basicTestEnso/ENSO_proc
netcdf_path: demo_output_enso_new_obs/basicTestEnso/ENSO_proc
debug: False
obs_cmor: True
obs_cmor_path: demo_data_enso_new_obs/obs4MIPs_PCMDI_monthly
egg_pth: /opt/anaconda3/envs/pmp_20220321/share/pmp
output directory for graphics:demo_output_enso_new_obs/basicTestEnso/ENSO_proc
output directory for diagnostic_results:demo_output_enso_new_obs/basicTestEnso/ENSO_proc
output directory for metrics_results:demo_output_enso_new_obs/basicTestEnso/ENSO_proc
list_variables: ['ssh', 'sst', 'taux', 'thf']
list_obs: ['AVISO-1-0', 'ERA-INT', 'GPCP-2-3', 'HadISST-1-1']
Observation dataset AVISO-1-0 is not given for variable thf
Observation dataset GPCP-2-3 is not given for variable thf
Observation dataset HadISST-1-1 is not given for variable thf
PMPdriver: dict_obs readin end
Process start: Wed Mar 30 10:36:29 2022
models: ['ACCESS1-0']
 ----- model:  

                    "areacella",
                    "areacella"
                ],
                "landmaskname": [
                    "sftlf",
                    "sftlf",
                    "sftlf",
                    "sftlf",
                    "sftlf",
                    "sftlf"
                ],
                "path + filename": [
                    "demo_data_enso_new_obs/CMIP5_demo_data/hfls_Amon_ACCESS1-0_historical_r1i1p1_185001-200512.nc",
                    "demo_data_enso_new_obs/CMIP5_demo_data/hfss_Amon_ACCESS1-0_historical_r1i1p1_185001-200512.nc",
                    "demo_data_enso_new_obs/CMIP5_demo_data/rlds_Amon_ACCESS1-0_historical_r1i1p1_185001-200512.nc",
                    "demo_data_enso_new_obs/CMIP5_demo_data/rlus_Amon_ACCESS1-0_historical_r1i1p1_185001-200512.nc",
                    "demo_data_enso_new_obs/CMIP5_demo_data/rsds_Amon_ACCESS1-0_historical_r1i1p1_185001-200512.nc",
                    "demo_data_enso_new_obs/CMIP5_demo_data/rsus_Amo

INFO::2022-03-30 10:41::pcmdi_metrics:: Results saved to a json file: /Users/lee1043/Documents/Research/git/pcmdi_metrics_20220106_2/pcmdi_metrics/doc/jupyter/Demo/demo_output_enso_new_obs/basicTestEnso/ENSO_proc/cmip5_historical_ENSO_proc_basicTestEnso_ACCESS1-0_r1i1p1.json
INFO::2022-03-30 10:41::pcmdi_metrics:: Results saved to a json file: /Users/lee1043/Documents/Research/git/pcmdi_metrics_20220106_2/pcmdi_metrics/doc/jupyter/Demo/demo_output_enso_new_obs/basicTestEnso/ENSO_proc/cmip5_historical_ENSO_proc_basicTestEnso_ACCESS1-0_r1i1p1_diveDown.json


               EnsoUvcdatToolsLib AverageMeridional
               EnsoUvcdatToolsLib AverageMeridional
     ComputeMetric: oneVarRMSmetric, BiasSstLonRmse = ACCESS1-0_r1i1p1 and HadISST
                         NOTE: Estimated landmask applied
               EnsoUvcdatToolsLib AverageMeridional
               EnsoUvcdatToolsLib AverageMeridional
     ComputeCollection: metric = BiasTauxLonRmse
     ComputeMetric: oneVarRMSmetric, BiasTauxLonRmse = ACCESS1-0_r1i1p1 and ERA-Interim
               EnsoUvcdatToolsLib AverageHorizontal
                         NOTE: Estimated landmask applied
               EnsoUvcdatToolsLib AverageHorizontal
                         NOTE: Estimated landmask applied
               EnsoUvcdatToolsLib AverageMeridional
               EnsoUvcdatToolsLib AverageMeridional
     ComputeCollection: metric = EnsoAmpl
     ComputeMetric: oneVarmetric = ACCESS1-0_r1i1p1
     ComputeMetric: oneVarmetric = ERA-Interim
                         NOTE: Estimated landmask

                         NOTE: Estimated landmask applied
               EnsoUvcdatToolsLib AverageMeridional
               EnsoUvcdatToolsLib AverageMeridional
     ComputeCollection: metric = EnsoSstSkew
     ComputeMetric: oneVarmetric = ACCESS1-0_r1i1p1
     ComputeMetric: oneVarmetric = ERA-Interim
                         NOTE: Estimated landmask applied
     ComputeMetric: oneVarmetric = HadISST
                         NOTE: Estimated landmask applied
PMPdriver: model loop end
Process end: Wed Mar 30 10:41:12 2022
